### rankings_payoffs.ipynb

- Egalitarianism minimises the disadvantage of the worst off agent
    - Utilitarianism is the maximisation of total utility

- The purpose of rankings here is to remove the extent to which you prefer one solution to another

This notebook will
- Normalise the the rankings of a utilitarian solution in relation to an egalitarian one, and compare.

- Given an agents principles, create rankings based on how much they wish to be influenced by another agents rankings
    - A preferred ranking of their preferences considering their ethical principle

- Once an agents ranking has been computed, we compare the distance between their rankings, and the rankings of each consensus (Egal, Util, Transition, HCVA) to find closest.

#### Full Explanation for clarity
Each agent is comprised of a value system, and a principle preference
- The value system has, for every value, a preference value of each value over every other value. These are our preferences.
    - The value system also has, for every action, an action judgement for every value. The action judgement tells us how much each value in the system would be promoted or demoted if that action is taken.
- Additionally, the principle preference tells us the extent to which the agent prefers a principle over another principle. As there are only 2 principles this is a number between 0-1. 0 signfies full support for Utilitarianism, and 1.0 signifies full support for egalitarianism

Here the principle preference isnt simply a vote for one thing or another, its just a distance measure. When every agents principle preferences are collated, we optimise on them to find HCVA. We could always use another metric, such as MSE of the principle preferences.

To convert these distances to rankings, we need to check each agents principle preference (a number between 0 and 1), convert that preference to an actual principle value (between 1.0 and 10.0), and then rank the 4 options based on their distance from the agents real preferred principle value.
- This removes the extent one solution is preferred over the other
- This gives an accurate ranking of each agents preference for each solution

The files chosen for use here are the processed_data from the European Social Study (1 action, 2 values) where the values for each nation have been multipled by a preference factor of 2.5 and an action factor of 5.0 giving a transition point of 1.40 and HCVA point of 2.1 

In [60]:
# import needed files/libs
import pandas as pd

agent_data_path = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_one_action_ess.csv_with_factor_2.5_5.0.csv"
agent_data_df = pd.read_csv(agent_data_path)

consensus_data_path_pref = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/results/14-11-results-factor-2.5-5.0-prefs.csv"
consensus_data_path_act = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/results/14-11-results-factor-2.5-5.0.csv"
temp_pref = pd.read_csv(consensus_data_path_pref)
temp_act = pd.read_csv(consensus_data_path_act)
consensus_data_df = pd.merge(temp_pref, temp_act, on='p')

agent_principle_data_path = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/results/14-11-processed_data_with_principles_ess.csv"
agent_principle_data_df = pd.read_csv(agent_principle_data_path)
agent_principle_data_df.rename(columns={'rel': 'egal', 'nonrel': 'util'}, inplace=True)


In [61]:
# First create rankings of preferences for each consensus

consensus_rankings = {"util": 1.0, "egal": 10.0, "t": 1.4, "hcva": 2.1}

In [62]:
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    egal = row_data['egal']
    util = row_data['util']
    # Normalising means that egal is 0 and util is 1
    normalised_value = egal / (egal + util)
    agent_principle_data_df.at[index, 'normalised_value'] = normalised_value

# Now each row is normalised, we convert the normalised value into the principle value for each agent, between 1.0 and 10.0
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    normalised_value = row_data['normalised_value']
    principle_value = normalised_value * 9 + 1
    agent_principle_data_df.at[index, 'principle_value'] = round(principle_value, 2)

# Now calculate the ranking for of consensus rankings for each principle we are interested in
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    temp_distances = {}
    for principle, value in consensus_rankings.items():
        agent_principle = row_data['principle_value']
        temp_distances[principle] = abs(agent_principle - value)
    temp_distances = {key: value for key, value in sorted(temp_distances.items(), key=lambda item: item[1])}
    # Place in dataframe for agent
    for i, (principle, distance) in enumerate(temp_distances.items()):
        agent_principle_data_df.at[index, principle + "_rank"] = i + 1

agent_principle_data_df


,country,egal,util,normalised_value,principle_value,hcva_rank,t_rank,util_rank,egal_rank
0,AT,204.0,1654.0,0.109795,1.99,1.0,2.0,3.0,4.0
1,BE,834.0,903.0,0.480138,5.32,1.0,2.0,3.0,4.0
2,CH,143.0,1290.0,0.099791,1.90,1.0,2.0,3.0,4.0
3,CZ,309.0,1751.0,0.150000,2.35,1.0,2.0,3.0,4.0
4,DE,440.0,2312.0,0.159884,2.44,1.0,2.0,3.0,4.0
5,EE,192.0,1728.0,0.100000,1.90,1.0,2.0,3.0,4.0
6,ES,256.0,1454.0,0.149708,2.35,1.0,2.0,3.0,4.0
7,FI,496.0,1342.0,0.269859,3.43,1.0,2.0,3.0,4.0
8,FR,216.0,1746.0,0.110092,1.99,1.0,2.0,3.0,4.0
9,GB,292.0,1536.0,0.159737,2.44,1.0,2.0,3.0,4.0


### Display results in LaTeX format

In [63]:
# Calculate the frequency of each strategy at each rank
rank_frequencies = agent_principle_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].apply(pd.Series.value_counts).fillna(0).astype(int)

# Create a LaTeX table
latex_rank_frequencies = rank_frequencies.T.to_latex()

print(latex_rank_frequencies)

\begin{tabular}{lrrrr}
\toprule
 & 1.000000 & 2.000000 & 3.000000 & 4.000000 \\
\midrule
hcva_rank & 18 & 5 & 0 & 0 \\
t_rank & 0 & 18 & 5 & 0 \\
util_rank & 0 & 0 & 18 & 5 \\
egal_rank & 5 & 0 & 0 & 18 \\
\bottomrule
\end{tabular}



In [64]:
# Sum up the ranks for each principle
rank_sums = agent_principle_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].sum()

# Create a LaTeX table
latex_table = rank_sums.to_frame(name='Sum of Ranks').T.to_latex()

print(latex_table)

\begin{tabular}{lrrrr}
\toprule
 & hcva_rank & t_rank & util_rank & egal_rank \\
\midrule
Sum of Ranks & 28.000000 & 51.000000 & 74.000000 & 77.000000 \\
\bottomrule
\end{tabular}

